In [58]:
import pandas as pd
from gensim import corpora
import gensim
import pyLDAvis.gensim
import pyLDAvis
import os

In [59]:
df = pd.read_csv('data/articles.csv')
df['lemmas'] = df['lemmas'].map(eval)

In [60]:
dictionary = corpora.Dictionary(df['lemmas'].tolist())
dictionary.save('data/gensim/dictionary.dict')

In [61]:
df['corpus'] = df['lemmas'].map(dictionary.doc2bow)
corpora.MmCorpus.serialize('data/gensim/corpus.mm', df['corpus'].tolist())
corpus = gensim.corpora.MmCorpus('data/gensim/corpus.mm')

## LDA

In [62]:
n_topics = 40

model_name = 'lda_{}'.format(n_topics)
model_folder = os.path.join(os.path.abspath(os.curdir), 'models', model_name)

In [63]:
lda = gensim.models.ldamulticore.LdaMulticore(corpus=corpus, id2word=dictionary, num_topics=n_topics)

In [64]:
if not os.path.exists(model_folder):
    os.mkdir(model_folder)
lda.save(os.path.join(model_folder, model_name + '.model'))

In [65]:
with open(os.path.join(model_folder, model_name + '.html'), mode='w') as f:
    pyLDAvis.save_html(pyLDAvis.gensim.prepare(lda, corpus=corpus, dictionary=dictionary), f)